# Preprocess all SEHNAMI data

Data downloaded from meteodat database on 02.03.2020. Data goes up to august 2019. 

In [ ]:
%matplotlib inline

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import salem
import hvplot.pandas
import holoviews as hv
import geoviews as gv
import geopandas as gpd
import cartopy.crs as ccrs
import shapely.geometry as shpg
hv.extension('bokeh')

In [ ]:
# parameters
processed_dir = './processed/'
file_name = 'precip'
buffer_size = 40 # in km

In [ ]:
var_names = {'precip':'RR', 'mean_temp':'TMEAN', 'max_temp':'TMAX', 'min_temp':'TMIN' }
if file_name not in var_names:
    raise ValueError('File name: {} not recognized'.format(file_name))
var_name = var_names[file_name]

## Raw data to metadata

In [ ]:
def count_achtungs(f, do_print=False):
    with open(f, 'r') as file:
        n_achtungs = 0
        achtungs = []
        while True:
            l = file.readline()
            if 'ACHTUNG' in l:
                n_achtungs += 1
                achtungs.append(l)
            else:
                break
    if do_print:
        print('Filename ', f)
        print('Number of Achtungs: ', n_achtungs)
        print('Achtungs:')
        for a in achtungs:
            print('  ' + a)
        print('')
    return n_achtungs

In [ ]:
f = './raw/SEHNAMI/{}_all.csv'.format(file_name)
n_achtungs = count_achtungs(f, do_print=True)
df_meta = pd.read_csv(f, sep=';', index_col=0, skiprows=n_achtungs, nrows=5, header=None)
df_meta = df_meta.T
df_meta.columns = [c.strip().replace('DATE', 'ID') for c in df_meta.columns]
df_meta['ID'] = df_meta['ID'].map(lambda x: x.strip().replace('_{}'.format(var_name), ''))
df_meta = df_meta.set_index('ID')
df_meta.columns = [c.strip().replace('DATE', 'ID') for c in df_meta.columns]
df_meta['NAME'] = [c.strip() for c in df_meta['NAME']]
df_meta['LAT'] = df_meta['LAT'].astype(float)
df_meta['LON'] = df_meta['LON'].astype(float)
df_meta['HEIGHT'] = df_meta['HEIGHT'].astype(float)
df_meta.to_csv(processed_dir + 'L0_{}_StationsMeta.csv'.format(file_name))

In [ ]:
df_meta.head()

## L1 data: RAW data and coarse filter for stations with at least 1 year of data in the period 1980-2018

In [ ]:
df = pd.read_csv(f, sep=';', index_col=0, header=4+n_achtungs, parse_dates=True,
                 date_parser=lambda x: pd.datetime.strptime(x, '%d.%m.%Y'))
df.columns = [c.strip().split('_')[0] for c in df.columns]
df = df.replace(-9999, np.NaN)

In [ ]:
df.tail()

In [ ]:
# Sum up the valid days
df_s = (~ df.loc['1980':].isnull()).sum()
# Plot the number of stations with at least X year of data
n_stats = pd.DataFrame()
for ny in range(31):
    n_stats.loc[ny, 'N_STATIONS'] = len(df_s.loc[df_s >= (365*ny)])
n_stats.index.name = 'N_YEARS'
n_stats.hvplot(ylim=(0, int(n_stats.max()+100)), title='Number of stations with at least X year of data')

In [ ]:
# Select over a year
df_s = df_s.loc[df_s >= 365]
df = df[df_s.index]
df_meta = df_meta.loc[df.columns]

In [ ]:
n_obs = (~ df.isnull()).sum(axis=1)
n_obs.hvplot(ylim=(0, int(n_obs.max()+20)), title='Number of daily valid observations')

In [ ]:
df.to_csv(processed_dir + 'L1_{}_all_stats.csv'.format(file_name))

In [ ]:
n_years = (~ df.isnull()).sum(axis=0) / 365
cat = pd.cut(n_years, [0, 5, 10, 20, 30, 70], labels=['< 5', '< 10', '< 20', '< 30', '> 30'])
dfm = df_meta.loc[cat.index].copy()
dfm['N_YEARS'] = cat

graph = dfm.sort_values(by='N_YEARS').hvplot.points(x='LON', y='LAT', c='N_YEARS', 
                                                    hover_cols=['NAME', 'HEIGHT'], 
                                                    geo=True) 
graph = graph.options(width=600, height=600,  active_tools=['pan', 'wheel_zoom'])
graph * gv.tile_sources.StamenTerrain

## L2: Data within a buffer region around riosanta

In [ ]:
gdf_riosan = gpd.read_file('../gis/RioSanta/riosan_sel_one.shp')
gdf_riosan_buf = gpd.GeoDataFrame(gdf_riosan.buffer(buffer_size*1e3).to_frame(name='geometry'))
gdf_riosan_buf.crs = gdf_riosan.crs
gdf_riosan_buf_ll = gdf_riosan_buf.to_crs('+proj=longlat +datum=WGS84 +no_defs')
gdf_riosan = gdf_riosan.append(gdf_riosan_buf, sort=False)

In [ ]:
# GeoDataframe of station points
gdf_meta = gpd.GeoDataFrame(df_meta).copy()
gdf_meta['geometry'] = [shpg.Point(lon, lat) for (lon, lat) in zip(gdf_meta.LON, gdf_meta.LAT)]
# Test into
gdf_meta['is_in'] = gdf_meta.intersects(gdf_riosan_buf_ll.unary_union)

In [ ]:
# Select data
df_meta = df_meta.loc[gdf_meta.is_in]
df = df[df_meta.index]

In [ ]:
# Sum up the valid days
df_s = (~ df.loc['1980':].isnull()).sum()
# Plot the number of stations with at least X year of data
n_stats = pd.DataFrame()
for ny in range(31):
    n_stats.loc[ny, 'N_STATIONS'] = len(df_s.loc[df_s >= (365*ny)])
n_stats.index.name = 'N_YEARS'
n_stats.hvplot(ylim=(0, int(n_stats.max()+10)), title='Number of stations with at least X year of data')

In [ ]:
# Select over a year
df_s = df_s.loc[df_s >= 365]
df = df[df_s.index]
df_meta = df_meta.loc[df.columns]

In [ ]:
n_obs = (~ df.isnull()).sum(axis=1)
n_obs.hvplot(ylim=(0, int(n_obs.max()+5)), title='Number of daily valid observations')

In [ ]:
df.to_csv(processed_dir + 'L2_{}_all_stats.csv'.format(file_name))

In [ ]:
n_years = (~ df.isnull()).sum(axis=0) / 365
cat = pd.cut(n_years, [0, 5, 10, 20, 30, 70], labels=['< 5', '< 10', '< 20', '< 30', '> 30'])
dfm = df_meta.loc[cat.index].copy()
dfm['N_YEARS'] = cat

graph = dfm.sort_values(by='N_YEARS').hvplot.points(x='LON', y='LAT', c='N_YEARS', 
                                                    hover_cols=['NAME', 'HEIGHT'], 
                                                    geo=True) 
graph = graph.options(width=600, height=600,  active_tools=['pan', 'wheel_zoom'])
poly = gv.Polygons(gdf_riosan.to_crs(ccrs.GOOGLE_MERCATOR.proj4_init), crs=ccrs.GOOGLE_MERCATOR)
graph * gv.tile_sources.StamenTerrain * poly.opts(color='black', fill_color=None, width=600, height=600)

## L3: with at least 10 years of data within buffer after 2000

In [ ]:
# Sum up the valid days
df_s = (~ df.loc['2000':].isnull()).sum()
# Select over N years
N = 10
df_s = df_s.loc[df_s >= N * 365]
df = df[df_s.index]
df_meta = df_meta.loc[df.columns]

In [ ]:
df.to_csv(processed_dir + 'L3_{}_all_stats.csv'.format(file_name))

In [ ]:
df_meta['AVG'] = df.mean()

graph = df_meta.hvplot.points(x='LON', y='LAT', c='AVG', 
                              hover_cols=['index', 'NAME', 'HEIGHT', 'AVG'], 
                              geo=True) 
graph = graph.options(width=600, height=600,  active_tools=['pan', 'wheel_zoom'])
poly = gv.Polygons(gdf_riosan.to_crs(ccrs.GOOGLE_MERCATOR.proj4_init), crs=ccrs.GOOGLE_MERCATOR)
graph * gv.tile_sources.StamenTerrain * poly.opts(color='black', fill_color=None, width=600, height=600)